# **Carga inicial y librerias**

Primero, dejaremos los instaladores de algunas librerias que a lo mejor no vienen por defecto en este notebook

In [ ]:
!pip install surprise

In [ ]:
!pip install pandas

Se hace la carga inicial y se ven los tipos de datos del dataset. Muy importante, ya que a la hora de operar con algunos hay que hacer conversiones y tratamientos de formato.

In [ ]:
import pandas as pd
import ast
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import SVD, Dataset, Reader

import warnings; warnings.simplefilter('ignore')

#Hacer el load inicial
# movies = pd.read_csv('tmdb_5000_movies.csv')
movies = pd.read_csv('/content/drive/MyDrive/Cuarto/abp_proyecto/TMDB/tmdb_5000_movies.csv')

print(movies.info())

# **Asociar géneros a peliculas de manera más intuitiva**

Una operación común en un sistema de recomendación de este tipo es el filtrado por género de peliculas, por esa razón queremos construir un pequeño dataframe a partir del original que asigne de una forma mucho más intuitiva los generos a sus determinadas pelis. Lo que genere este código podría facilitar la puesta en el navegador

In [ ]:
# por practicidad usaremos para el ejemplo sólo 5 pelis
cabeza = movies.head() 

titles = cabeza["title"]
genres = movies["genres"]

# usamos un set, para evitar las repeticiones
generos = set()

# metemos al set los distintos generos
for i in range(len(genres)):
 salida = ast.literal_eval(genres[i])
 for j in range(len(salida)):
  generos.add(salida[j]["name"])

# la convertimos a lista para que su manejo sea más 
# facil a la hora de armar el DF
generos = list(generos) 
generos = sorted(generos) 

# creamos el nuevo dataframe con nuestras pelis como 
# filas y los géneros como columnas
df = pd.DataFrame(index=titles ,columns=generos)

df



Ya tenemos el Data frame como lo queriamos, ahora lo que quedaría por hacer es rellenarlo con valores correctos, aplicando un poco de lógica sobre los valores que tenemos en el Dataset original

In [ ]:
# aplicamos la longitud del dataframe 
# para que no se salga de rango

for i in range(len(df)):
  literales = ast.literal_eval(genres[i])
  generos_para_peli = set()

  for j in range(len(literales)):
    generos_para_peli.add(literales[j]['name'])
  
  generos_para_peli = list(generos_para_peli)

  for genero in generos_para_peli:
    df.iloc[i][genero] = 1
    
#poner el resto de columnas que están en NaN a 0
  for column in df.columns:
    if(np.isnan(df.iloc[i][column])):
      df.iloc[i][column] = 0

df

# **Recuperar los datos más relevantes de las peliculas**

Otra operación común es la recuperación de la descripción corta(overview), fecha de estreno, idioma original y página web de la pelicula

In [ ]:
# Se usa la misma variable reducida para los titulos 

release_dates = movies["release_date"]
overviews = movies["overview"]
homepages = movies["homepage"]
languages = movies["original_language"]

df_movies = pd.DataFrame(index=titles, columns=["overview",
                                                "release date", 
                                                "homepage", 
                                                "original language"])

# aplicamos el mismo principio de antes para poner a cada peli sus datos 

for i in range(len(df_movies)):
  df_movies.iloc[i]["overview"] = overviews[i]
  df_movies.iloc[i]["release date"] = release_dates[i]
  df_movies.iloc[i]["homepage"] = homepages[i]
  df_movies.iloc[i]["original language"] = languages[i]

df_movies

# **Recomendaciones basadas en la descripción**

Usando las descripciones se relacionarán varias películas para mostrarles al usuario. Haremos la carga de otro Dataset de películas que es un poco más completo y contiene un montón de datos importantes para nuestro sistema de recomendación

In [ ]:
# hacemos la carga del dataset

# mv_metadata = pd.read_csv('movies_metadata.csv')
mv_metadata = pd.read_csv('/content/drive/MyDrive/Cuarto/abp_proyecto/TMDB/archive/movies_metadata.csv')
mv_metadata.head()


Para realizar las operaciones también haremos uso de un dataset auxiliar, el cual contiene las relaciones por id de las peliculas en dos datasets distintos

In [ ]:
# links_small = pd.read_csv('links_small.csv')
links_small = pd.read_csv('/content/drive/MyDrive/Cuarto/abp_proyecto/TMDB/archive/links_small.csv')

# realizamos una conversion de tipos
# ya que los valores de interes están como string
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

mv_metadata = mv_metadata.drop([19730, 29503, 35587])

mv_metadata['id'] = mv_metadata['id'].astype('int')

# tenemos el dataframe con los datos relacionados
small_mv_metadata = mv_metadata[mv_metadata['id'].isin(links_small)]

# links_small.head()

small_mv_metadata.head(3) # visualizar que datos tiene


Para construir relaciones entre una pelicula dada usaremos los atributos 'description' y 'tagline' del dataset o dataframe que hemos construido antes. A continuación, usaremos la función para la similaridad del coseno

In [ ]:
small_mv_metadata['tagline'] = small_mv_metadata['tagline'].fillna('')
small_mv_metadata['description'] = small_mv_metadata['overview'] + small_mv_metadata['tagline']
small_mv_metadata['description'] = small_mv_metadata['description'].fillna('')

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(small_mv_metadata['description'])

small_mv_metadata = small_mv_metadata.reset_index()
titles = small_mv_metadata['title']
indices = pd.Series(small_mv_metadata.index, index=small_mv_metadata['title'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


Por último, para que sea mucho más sencillo el uso se creará una funcion para obtener las recomendaciones basadas en el título pasado como argumento.

In [ ]:
def recomendaciones_de(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

recomendaciones_de("Avatar").head(10)

## **Escoger películas por popularidad y ratings**

Ahora no sólo recomendará peliculas que coincidan basadas en la descripción, sino que escogerá de esa relación cual tienen un rating más alto y mayor popularidad, generando una selección filmográfica mucho más completa y de mejor calidad.

Utilizamos las clasificaciones de TMDB para crear nuestra tabla de películas principales. Usaremos la fórmula de calificación ponderada de IMDB para construir nuestro gráfico. Matemáticamente, se representa de la siguiente manera:

Calificación ponderada (WR) = (vv + m.R) + (mv + m.C)
dónde,

-v es el número de votos para la película

-m son los votos mínimos requeridos para aparecer en la tabla

-R es la calificación promedio de la película

-C es el voto medio de todo el informe

Ahora tenemos que determinar un valor apropiado para m, los votos mínimos requeridos para aparecer en la tabla. Usaremos el percentil 95 como nuestro límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 95% de las películas de la lista.

In [ ]:
vote_counts = mv_metadata[mv_metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = mv_metadata[mv_metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.95)

def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def pelis_popularidad(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    #aqui viene el código nuevo que mejora a nuestra
    #funcion anterior
    pelis = small_mv_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date']]
    vote_counts = pelis[pelis['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = pelis[pelis['vote_average'].notnull()]['vote_average'].astype('int')

    vote_counts = pelis[pelis['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = pelis[pelis['vote_average'].notnull()]['vote_average'].astype('int')
    
    qualified = pelis[(pelis['vote_count'] >= m) & (pelis['vote_count'].notnull()) & (pelis['vote_average'].notnull())]

    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

pelis_popularidad('The Dark Knight')

Tambien aplicando las formulas anteriores podemos crear la función para que nos dé las películas por genero basadas también en su popularidad, eso ofrece aún más juego a la hora de recomendar a los usuarios. Lo que haremos en esta es bajar el percentil de 0.95 a 0.85

In [ ]:
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
salida = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
salida.name = 'genre'
genero_pelis = movies.drop('genres', axis=1).join(s)

def popularidad_y_genero(genre, percentile=0.85):
    df = genero_pelis[genero_pelis['genre'] == genre]
    votos = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    promedio = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = promedio.mean()
    m = votos.quantile(percentile)
    
    # creamos nuestro dataframe de notas a partir del genero
    nota = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 
                                                                                                        'year', 
                                                                                                        'vote_count', 
                                                                                                        'vote_average', 
                                                                                                        'popularity']]
    nota['vote_count'] = nota['vote_count'].astype('int')
    nota['vote_average'] = nota['vote_average'].astype('int')
    nota['wr'] = nota.apply(weighted_rating, axis=1)
    nota = nota.sort_values('wr', ascending=False).head(250)
    
    return nota

popularidad_y_genero('Drama').head(15)

# **Recomendaciones por usuario**

Se usarán técnicas de filtrado híbrido, que combinan el filtrado colaborativo, para tener mucha más personalización, y las recomendaciones basadas en contenido. La función aceptará como argumentos un id de usuario y una pelicula, y mediante las técnicas antes mencionadas, aunque la película sea la misma, si el usuario es diferente se recomendarán películas distintas.

In [ ]:
# parte del filtrado colaborativo

# ratings = pd.read_csv('ratings_small.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Cuarto/abp_proyecto/TMDB/archive/ratings_small.csv')
svd = SVD()
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd.fit(trainset)
id_map = pd.read_csv('/content/drive/MyDrive/Cuarto/abp_proyecto/TMDB/archive/links_small.csv')[['movieId', 'tmdbId']]


#funciones hibridas para la parte hibrida
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(small_mv_metadata[['title', 'id']], on='id').set_index('title')

indices_map = id_map.set_index('id')

def recomendacion_hibrida(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    pelis = small_mv_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    pelis['est'] = pelis['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    pelis = pelis.sort_values('est', ascending=False)
    return pelis.head(10)



Aunque sea para una misma pelicula al ser un usuario con un id diferente se recomiendan pelis distintas

In [ ]:
#probamos para el usuario 1
recomendacion_hibrida(1, 'The Dark Knight')

In [ ]:
# y ahora vemos la salida para el usuario 30, por ejemplo
recomendacion_hibrida(30, 'The Dark Knight')